Intro to supervised ML.
Imports needed first:

In [2]:
# Libraries to manage data
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split


# Imports to visualise data
import matplotlib as plt
import seaborn as sns
from matplotlib import pyplot

# Libraries for the classifier

from sklearn.tree import DecisionTreeClassifier # for decision trees
from sklearn.neighbors import KNeighborsClassifier # for K nearest neighbours
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# for checks
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay

# for time
import timeit


After importing all of the modules, we can begin by grabbing our dataset and cleaning it before doing operaations.

In [5]:
b_cancer = datasets.load_breast_cancer() 
dataset = pd.DataFrame(data=b_cancer.data, columns=b_cancer.feature_names)
dataset["target"] = b_cancer.target
print(dataset["target"])


0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64


For now, we will detect any N/A data and simply remove them.

In [7]:
print("Shows NAs per column!")
dataClean = dataset.dropna(axis=0, how="any")
print(dataClean.isna().sum())

Shows NAs per column!
mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64
